## We will give tour of most prominent decoding models, mainly Greedy search, Beam search, Top-K sampling and Top P

In [16]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==2.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.1


In [17]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Add Tokenizer
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')

# Add EOS token as PAD to avoid warnings
model=TFGPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


# Greedy Search-
* Greedy search is a simple text generation method where at each time step, the model selects the highest probability token as the next word. It does not look ahead

In [18]:
# Encode the context
input_ids=tokenizer.encode('I enjoy walking with my cute dog',return_tensors='tf')

# Generate the output
greedy_output=model.generate(input_ids,max_length=50)

print(tokenizer.decode(greedy_output[0],skip_special_tokens=True))

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


The biggest drawbask of greedy search,is it misses the high probability words hidden behind low probability words. So it starts repeting the words.


# Beam Search -
Beam search reduce the risk of missing hidden high probability word sequence by keeping most likely `num_beams` of hypotheses at each time step and eventually choosing the hypothesis that has the overall high probability.

In [19]:
beam_output=model.generate(input_ids,max_length=50,num_beams=5,early_stopping=True)

print(tokenizer.decode(beam_output[0],skip_special_tokens=True))

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


So as we can see our model repeat the sentence again and again. So we are going to use n-gram penalties.N-gram penalties are teh mechanism to prevent repetitive text generation by penalizing the model if it tries to generate the same same N-gram more than once.


In [20]:
output_generate=model.generate(input_ids,max_length=50,num_beams=5,no_repeat_ngram_size=2,early_stopping=True)

print(tokenizer.decode(output_generate[0],skip_special_tokens=True))

I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


So we have `num_return_sequences` which will help us to determines how many of the best beams are returned at the end. It is used with combination of `num_beams`- which determines how many search paths are explored during text generation.


In [21]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to get back to
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about 

So the problem over is that as we seen that it generates the output according to the high probability of the token  so as human being we do not expect the output like that , which will generate on probabilities, we want output like we do not expect the common output . so we use the sampling method where we select randomly next token with conditional probabilities

# Sampling-
Sampling means randomly picking next token according to conditional probability distribution.

In [22]:
tf.random.set_seed(0)

sample_output=model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
    )
print(tokenizer.decode(sample_output[0],skip_special_tokens=True))

I enjoy walking with my cute dog and never seem to get nervous until it turns into battery Art Hours $50 of Weekend Bikes Friday Night A King's Approaches This was all, no problem WRONG!! WOWMS readers: excellent product!


* So there is risk of incoherent text. Model might generate sentence that do not make sense grammatically
* To generate output sharper we use the `temperature`

In [23]:
sample_output=model.generate(
    input_ids,
    max_length=50,
    do_sample=True,
    top_k=0,
    temperature=0.7
)
print(tokenizer.decode(sample_output[0],skip_special_tokens=True))

I enjoy walking with my cute dog. He's just a little quiet and quiet. He's so hungry. He loves puppies, so we're all so so excited to have him. We'll be loving him all the time."

Dr.


# Top-K sampling-
Model chooses the next token from top `k` most probable tokens. The model first calculate the probability of all tokens. Only the top `k` tokens with the highest probabilites are kept.
* This introduce creativity and diversity because the next token is chosen randomly.
* It avoids the predictible and repetative words as we can see in greedy search .


In [24]:
sample_output=model.generate(
          input_ids,
          max_length=50,
          do_sample=True,
          top_k=50

)
print(tokenizer.decode(sample_output[0],skip_special_tokens=True))

I enjoy walking with my cute dog. He won't touch me like that! He's a little timid. He makes me feel very small in my home. Maybe it's possible for me not to do this with a puppy but he is a nice


## So there is some problem in the Top-K sampling
* Top-k always considers a fixed number of most probable tokens.
* This is static, which means it does not adapt the context of the text.


### **Top-p (nucleus) sampling**

Instead of sampling only from the most likely *K* words, in *Top-p* sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability *p*.

In [25]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and very often see her immediately excited at moving on." - Alicia Davis

Recently, running around Los Angeles on an expeditions had me thinking, "What'd I get for free here?" I worked as a


In [26]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog

In my life I love having fun

What does it cost to feed a cat to get a heart attack?

I think, this is actually a really good question: what are some of the
1: I enjoy walking with my cute dog as much as I do driving my car. It's the best way to spend a Saturday without missing a beat.

5/5 - My favorite weekend in my life. The weekend I really wanted is Tuesday
2: I enjoy walking with my cute dog, I like to get a dog from the grocery store and it's been a pleasure to sit and talk to her (when she is sleeping), and she's been so happy with our visit to the supermarket (we
